In [1]:
# !pip install autogen
# !pip install openai

# or pip install -r requirements.txt (from the github repo)

In [1]:
# source: https://github.com/microsoft/autogen/blob/main/notebook/autobuild_agent_library.ipynb
config_file_or_env = './OAI_CONFIG_LIST'
default_llm_config = {
    'temperature': 0
}

In [2]:
# Create AgentBuilder instance using config path & default config
# Specify  builder & agent models
from autogen.agentchat.contrib.agent_builder import AgentBuilder
builder = AgentBuilder(
    config_file_or_env=config_file_or_env, 
    builder_model='gpt-4o')

flaml.automl is not available. Please install flaml[automl] to enable AutoML functionalities.


# VM suggestion

## How do I update a field in a database and then, use that value in a task to compare with another 'field-value' and based on the comparison, invoke a REST API, fetch the json and parse that json to print out json fields.

# SD use case: I have use case for healthcare which is to support collaborative assessment between drug sponsor, collaborative assessment team, administrative checking, reviewer, report drafting, etc.



In [8]:
# Specify the building tasks
# define building taskwith description *& examples
# helps bild manager decide on agents
# example task: "Create a multi-agent system for task X"

# building_task = """Create a multi-agent system for this type of workflow: 
# 'How do I update a field in a database and then, use that value in a task to compare with another 'field-value' and based on the comparison, invoke a REST API, fetch the json and parse that json to print out json fields.'
# ."""

building_task = """Build a multi-agent system for these profiles: 'administrative checking agent assigns incoming drug applications to the collaborative assessment team, which then collaborates to produce a draft report for a reviewer. The framework could automate communication, 
task delegation, and report drafting, allowing each agent to fulfill its role in the process.'"""

In [9]:
# use build() method to generate agents
# include a user proxy for tasks involving coding

agent_list, agent_configs = builder.build(building_task, default_llm_config, coding=True)

==> Generating agents...
['MultiAgentSystems_Expert', 'DrugApplication_Expert', 'Automation_Expert', 'TaskDelegation_Expert', 'ReportDrafting_Expert'] are generated.
==> Generating system message...
Preparing system message for MultiAgentSystems_Expert
Preparing system message for DrugApplication_Expert
Preparing system message for Automation_Expert
Preparing system message for TaskDelegation_Expert
Preparing system message for ReportDrafting_Expert
==> Generating description...
Preparing description for MultiAgentSystems_Expert
Preparing description for DrugApplication_Expert
Preparing description for Automation_Expert
Preparing description for TaskDelegation_Expert
Preparing description for ReportDrafting_Expert
==> Creating agents...
Creating agent MultiAgentSystems_Expert...
Creating agent DrugApplication_Expert...
Creating agent Automation_Expert...
Creating agent TaskDelegation_Expert...
Creating agent ReportDrafting_Expert...
Adding user console proxy...


In [10]:
agent_list

In [ ]:
agent_configs

{'building_task': "Build a multi-agent system for these profiles: 'administrative checking agent assigns incoming drug applications to the collaborative assessment team, which then collaborates to produce a draft report for a reviewer. The framework could automate communication, \ntask delegation, and report drafting, allowing each agent to fulfill its role in the process.'",
 'agent_configs': [{'name': 'MultiAgentSystems_Expert',
   'model': [],
   'tags': [],
   'system_message': '## Your role\nMultiAgentSystems_Expert is specialized in designing and implementing multi-agent systems, particularly for complex workflows that require coordination and communication between various agents.\n\n## Task and skill instructions\n- The task involves building a multi-agent system specifically for administrative drug application processing. The system should be capable of assigning incoming drug applications to a collaborative assessment team, facilitating collaboration for producing a draft repo

In [14]:
# Execute the task
# agents collaborate in a group chat to complete task
# combine llms, human inputs, and tools
# example: start_task(execution_task, agent_list, llm_config)
from autogen import config_list_from_json, GroupChat, GroupChatManager

config_list = config_list_from_json(config_file_or_env)

def start_task(execution_task: str, agent_list: list):
    group_chat = GroupChat(agents=agent_list, messages=[], max_round=12)
    manager = GroupChatManager(groupchat=group_chat, llm_config={"config_list": config_list, **default_llm_config})
    agent_list[0].initiate_chat(manager, message=execution_task)


start_task(execution_task="""In the post-market surveillance phase of a newly released drug, a healthcare provider reports a severe adverse event (AE) through an online portal. 
           The Data Entry Agent receives the report, validates that all necessary fields are completed, and passes it to the Triage Agent, which classifies the event 
           as high-priority due to its severity. The case is then assigned to the Medical Reviewer Agent, which collaborates with other specialized agents 
           (e.g., cardiology and toxicology) to assess the potential link between the drug and the adverse event. Once the analysis is complete, the Regulatory 
           Reporting Agent automatically generates a report for submission to regulatory authorities like the FDA, ensuring compliance with the required timeline. 
           Finally, the Administrative Agent tracks the reports submission and schedules a follow-up with the healthcare provider to gather any additional information.""",
    agent_list=agent_list,
)

Database_Expert (to chat_manager):

In the post-market surveillance phase of a newly released drug, a healthcare provider reports a severe adverse event (AE) through an online portal. 
           The Data Entry Agent receives the report, validates that all necessary fields are completed, and passes it to the Triage Agent, which classifies the event 
           as high-priority due to its severity. The case is then assigned to the Medical Reviewer Agent, which collaborates with other specialized agents 
           (e.g., cardiology and toxicology) to assess the potential link between the drug and the adverse event. Once the analysis is complete, the Regulatory 
           Reporting Agent automatically generates a report for submission to regulatory authorities like the FDA, ensuring compliance with the required timeline. 
           Finally, the Administrative Agent tracks the reports submission and schedules a follow-up with the healthcare provider to gather any additional information.

KeyboardInterrupt: 

In [8]:
builder.clear_all_agents()

All agents have been cleared.


In [9]:
saved_path = builder.save()

Building config saved to ./save_config_278c3bde745d76536b39f7f5952c1d5a.json


Configurations will be saved in JSON format like such:

```
// FILENAME: save_config_TASK_MD5.json
{
"building_task": "Find a paper on arxiv about artificial intelligence, 
and analyze its application in some domain. For example, find a latest 
paper about gpt-4 on arxiv and find its potential applications in 
software." ,
"agent_configs": [
{
    "name": "...",
    .....
},
...
],
"manager_system_message": "...",
"code_execution_config": {...},
"default_llm_config": {...}
}

```